In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

/opt/ml/anaconda3/envs/openmmlab/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

root='../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 64

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/faster_rcnn_r50_fpn_1x_trash'

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)

In [5]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [4]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [5]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2022-03-21 03:25:45,196 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2022-03-21 03:25:45,198 - mmcv - INFO - load model from: torchvision://resnet50
2022-03-21 03:25:45,200 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2022-03-21 03:25:45,412 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2022-03-21 03:25:45,446 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2022-03-21 03:25:45,474 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-03-21 03:25:45,484 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [6]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2022-03-21 03:25:54,434 - mmdet - INFO - Start running, host: root@65388b0842de, work_dir: /opt/ml/detection/baseline/mmdetection/work_dirs/faster_rcnn_r50_fpn_1x_trash
2022-03-21 03:25:54,435 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                

2022-03-21 03:29:07,785 - mmdet - INFO - Epoch [1][50/77]	lr: 1.978e-03, eta: 0:54:38, time: 3.751, data_time: 0.142, memory: 24147, loss_rpn_cls: 0.4259, loss_rpn_bbox: 0.0514, loss_cls: 0.6359, acc: 87.6443, loss_bbox: 0.1272, loss: 1.2404, grad_norm: 5.7235
2022-03-21 03:30:47,901 - mmdet - INFO - Saving checkpoint at 1 epochs
2022-03-21 03:33:56,601 - mmdet - INFO - Epoch [2][50/77]	lr: 5.055e-03, eta: 0:39:05, time: 3.725, data_time: 0.142, memory: 24152, loss_rpn_cls: 0.1311, loss_rpn_bbox: 0.0481, loss_cls: 0.3314, acc: 93.2507, loss_bbox: 0.2584, loss: 0.7690, grad_norm: 0.8045
2022-03-21 03:35:34,785 - mmdet - INFO - Saving checkpoint at 2 epochs
2022-03-21 03:38:44,120 - mmdet - INFO - Epoch [3][50/77]	lr: 8.132e-03, eta: 0:32:58, time: 3.738, data_time: 0.144, memory: 24152, loss_rpn_cls: 0.1000, loss_rpn_bbox: 0.0469, loss_cls: 0.3308, acc: 92.2229, loss_bbox: 0.3051, loss: 0.7828, grad_norm: 1.3461
2022-03-21 03:40:21,053 - mmdet - INFO - Saving checkpoint at 3 epochs
2022

learning_rate,▂▃▄▅▆▇██▂▂▂▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▁█▇▆▆▆▆▆▇▇▇▇
train/grad_norm,█▁▂▃▃▃▃▂▂▁▂▁
train/loss,█▃▃▃▃▂▂▂▁▁▁▁
train/loss_bbox,▁▆██▇▇▇▇▆▆▆▆
train/loss_cls,█▃▃▃▂▂▂▂▁▁▁▁
train/loss_rpn_bbox,█▇▆▅▄▄▃▃▁▁▁▁
train/loss_rpn_cls,█▃▂▂▂▁▁▁▁▁▁▁
learning_rate,0.0002
momentum,0.9
